In [1]:
import pandas as pd
import os
import numpy as np
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
from datetime import datetime


In [2]:
DATA_PATH = '../../Pokemon-data/'
NORMALIZED_DATA = '../../Training-baseline/'
SEED = 42
BATCH_SIZE = 64
epochs = 1
checkpoint_path = './saved-models/'
n_epochs_stop = 5

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
metadata = pd.read_csv('metadata/pokemon.csv')

pokemon_names = [x for x in os.listdir(NORMALIZED_DATA)]

filtered_metadata = metadata[metadata.name.isin(pokemon_names)]

filtered_list = filtered_metadata.loc[:,'name'].tolist()

filtered_metadata = filtered_metadata.loc[:,['name', 'type1']]

#fire_data = filtered_metadata[filtered_metadata["type1"] == 'fire']
#grass_data = filtered_metadata[filtered_metadata["type1"] == 'grass']
#filtered_metadata = pd.concat([fire_data, grass_data], ignore_index=True)

filtered_metadata.type1 = pd.Categorical(filtered_metadata.type1)
filtered_metadata['code'] = filtered_metadata.type1.cat.codes
print(filtered_metadata['code'])

def compile_training_data_to_list():
    all_data = []
    for pokemon in os.listdir(NORMALIZED_DATA):
        all_data += [pokemon + '/' + x for x in os.listdir(NORMALIZED_DATA + pokemon)]

    results = create_annotated_dataframe(all_data)
    return results

def create_annotated_dataframe(all_data):
    base_data = {'file_name': [], 'name': [], 'label': []}
    for item in all_data:
        if len(filtered_metadata[filtered_metadata['name'].str.contains(item.split('/')[0])]):
            base_data['file_name'].append(item)
            base_data['name'].append(item.split('/')[0])
            # yes, this is a bit ugly, but we have to match with the metadata
            base_data['label'].append(
                filtered_metadata[
                    filtered_metadata['name']==(item.split('/')[0])
                ].loc[:,'code'].tolist()[0])

    results, y_train = create_encoded_dataframe(base_data)
    return results, y_train

def create_encoded_dataframe(base_data):
    results = pd.DataFrame(base_data, columns = ['file_name', 'name', 'label'])
    labels = np.unique(results["label"])
    max_value = np.max(labels) + 1
    Y_train = np.eye(max_value)[results["label"]]
    return results, Y_train

0       9
1       9
2       9
3       6
4       6
       ..
714     7
715     4
716     1
718    15
720     6
Name: code, Length: 703, dtype: int8


In [4]:
encoded_data, y_train = compile_training_data_to_list()

In [5]:
class CustomDataset(Dataset):
	def __init__(self, x, y, img_dir):
		self.x = x
		self.y = y
		self.img_dir = img_dir
		self.classes = np.unique(self.y)


	def __len__(self):
		return len(self.x)


	def __getitem__(self, idx):
		img_path = os.path.join(self.img_dir, self.x[idx])
        # use the slice to remove a possible 4th alpha channel
		#image = iio.v2.imread(img_path)[:,:,:3]
		image = np.load(img_path)[:,:,:3]
		image = image.astype(np.float32)
		label = self.y[idx]
		return image, label

In [6]:
def stratified_split(dataset, labels):
    x_train, x_val, y_train, y_val = train_test_split(dataset['file_name'].to_numpy(),
                                                  labels,
                                                  test_size=0.25,
                                                  stratify=dataset['label'],
                                                  random_state=SEED)

    trainSet = CustomDataset(x_train, y_train, NORMALIZED_DATA)
    valSet = CustomDataset(x_val, y_val, NORMALIZED_DATA)
    return trainSet, valSet

In [7]:
train, val = stratified_split(encoded_data, y_train)
trainDataLoader = DataLoader(train, batch_size=BATCH_SIZE)
valDataLoader = DataLoader(val, batch_size=BATCH_SIZE)

In [8]:
from torchvision import models
model = models.efficientnet_b0(pretrained=True)

In [9]:
for param in model.parameters():
    param.requires_grad = False

In [10]:
import torch.nn as nn
# Add on classifier
model.classifier = nn.Sequential(
                      nn.Linear(1280, 256), 
                      nn.ReLU(), 
                      nn.Dropout(0.4),
                      nn.Linear(256, 18),                   
                      nn.LogSoftmax(dim=1))

In [11]:
model = model.to('cuda')

In [12]:
from torch import optim
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [13]:
for epoch in range(epochs):
    print(f"[INFO] starting training for epoch: {epoch + 1} at {datetime.now().strftime('%H:%M:%S')}")

    val_loss = 0
    min_val_loss = 0

    for data, targets in tqdm(trainDataLoader, desc="Training progress"):
        data = data.transpose(1, 3)
        data = data.transpose(2, 3)

        data = data.float()
        data, targets = (data.to(device), targets.to(device))
        # Generate predictions
        out = model(data)
        # Calculate loss
        loss = criterion(out, targets)
        # Backpropagation
        loss.backward()
        # Update model parameters
        optimizer.step()

    with torch.no_grad():
        model.eval()
        print(f"[INFO] starting evaluation for epoch: {epoch + 1} at {datetime.now().strftime('%H:%M:%S')}")
        for data, targets in tqdm(valDataLoader, desc="Validation progress"):
            data = data.transpose(1, 3)
            data = data.transpose(2, 3)

            data = data.float()
            data, targets = (data.to(device), targets.to(device))
            # Generate predictions
            out = model(data)
            # Calculate loss
            loss = criterion(out, targets)
            val_loss += loss

        # Average validation loss
        val_loss = val_loss / len(train)
        print(val_loss)
        # If the validation loss is at a minimum
        if val_loss < min_val_loss:
            # Save the model
            torch.save(model, checkpoint_path)
            epochs_no_improve = 0
            min_val_loss = val_loss

        else:
            epochs_no_improve += 1
            # Check early stopping condition
            if epochs_no_improve == n_epochs_stop:
                print('Early stopping!')

                # Load in the best model
                model = torch.load(checkpoint_path)

[INFO] starting training for epoch: 1 at 12:53:56


Training progress: 100%|██████████| 142/142 [03:00<00:00,  1.27s/it]


[INFO] starting evaluation for epoch: 1 at 12:56:57


Validation progress: 100%|██████████| 48/48 [00:20<00:00,  2.33it/s]


tensor(0.0125, device='cuda:0', dtype=torch.float64)


NameError: name 'min_val_loss' is not defined